## Load Libraries

In [35]:
import json
from random import randint, sample
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

import xgboost
import treelite


sns.set(style="ticks")
np.set_printoptions(suppress=True)

pd.set_option('display.max_rows', 150)

## Load Data

In [36]:
with open('/data/data/NonwearCheck/450/Results/annotations.json', 'r') as f:
    annotations = json.load(f)

record_annotation_index = {}
for record_annotation in annotations['record_annotations']:
    if record_annotation['id'] not in record_annotation_index:
        record_annotation_index[record_annotation['id']] = {}
    id = record_annotation['id']
    record_annotation.pop('id')
    record_annotation_index[id].update(record_annotation)

segment_annotation_index = {}
for segment_annotation in annotations['segment_annotations']:
    if segment_annotation['id'] not in segment_annotation_index:
        segment_annotation_index[segment_annotation['id']] = {}
    id = segment_annotation['id']
    segment_annotation.pop('id')
    segment_annotation_index[id].update(segment_annotation)

FileNotFoundError: [Errno 2] No such file or directory: '/data/data/NonwearCheck/450/Results/annotations.json'

In [45]:
df_feats = pd.read_csv("/data/data/NonwearCheck/456/Results/df_feat_ppg_gr.csv", index_col=None)
df_feats = df_feats.iloc[shuffle(range(len(df_feats)), random_state=0), :]

df_objects = pd.read_csv("/data/data/NonwearCheck/456/Results/df_object_ppg_gr.csv", index_col=None)

In [46]:
feats_cols = [c for c in df_feats.columns if "ppg" in c]
target_col = "wear_category_id"

## 全数据构建模型

In [55]:
# 利用xgboost选择前50个特征分析
params = {'max_depth': 6, 'objective': 'binary:logistic', "n_jobs": -1}
num_iter = 20

X_cols, y_col = [
    'ppg-g__ar_2',
    'ppg-g__number_peaks__n_3',
    'ppg-g__autocorrelation__lag_4',
    'ppg-g__agg_linear_trend__attr_"stderr"__chunk_len_5__f_agg_"var"',
    'ppg-g__agg_linear_trend__attr_"stderr"__chunk_len_10__f_agg_"mean"',
    'ppg-g__binned_entropy__max_bins_10',
    'ppg-g__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0',
    'ppg-g__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.8',
    'ppg-g__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',
    'ppg-g__ratio_beyond_r_sigma__r_1.5'
                       ], target_col

X, y = df_feats.loc[:, X_cols].values, df_feats.loc[:, y_col].values

# scaler = MinMaxScaler()
# X = scaler.fit_transform(X)

D_train = xgboost.DMatrix(X, label=y)
D_test = xgboost.DMatrix(X, label=y)

bst = xgboost.train(params, D_train, num_iter, [(D_train, 'train')], verbose_eval=False)

accuracy_score(y, bst.predict(D_test) > 0.6)

[00:11:16] WARNING: /xgboost/src/learner.cc:1043: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9844467108618052

In [40]:
model = treelite.Model.from_xgboost(bst)

model.export_srcpkg(platform='unix', toolchain='gcc', pkgpath='./model_20210526_003.zip',
                    libname='mymodel.so', verbose=True, params={'quantize': 1})

[00:00:27] /workspace/src/compiler/ast_native.cc:44: Using ASTNativeCompiler
[00:00:27] /workspace/src/compiler/ast/split.cc:24: Parallel compilation disabled; all member trees will be dumped to a single source file. This may increase compilation time and memory usage.
[00:00:27] /workspace/src/c_api/c_api.cc:286: Code generation finished. Writing code to files...
[00:00:27] /workspace/src/c_api/c_api.cc:291: Writing file recipe.json...
[00:00:27] /workspace/src/c_api/c_api.cc:291: Writing file header.h...
[00:00:27] /workspace/src/c_api/c_api.cc:291: Writing file main.c...


## 验证和C一致性

In [50]:
selected_object_ids = [5000]# list(range(3000, 3004))

### 待验证数据

In [64]:
selected_object_ids = [1000]
mask = np.isin(df_objects['id'], selected_object_ids)
mask = np.where(mask)[0]
df_objects.loc[mask, 'ppg-g'].values

array([1100.101, 1097.175, 1092.661, 1088.089, 1084.045, 1080.214,
       1076.103, 1073.449, 1072.673, 1074.012, 1073.969, 1073.602,
       1071.675, 1070.694, 1070.557, 1069.311, 1070.386, 1070.634,
       1072.527, 1072.4  , 1071.598, 1071.82 , 1071.853, 1068.97 ,
       1065.959, 1064.551, 1062.742, 1058.186, 1049.605, 1043.282,
       1038.496, 1036.765, 1041.739, 1050.747, 1060.61 , 1062.896,
       1066.684, 1064.764, 1059.509, 1053.178, 1044.878, 1036.663,
       1031.672, 1031.39 , 1030.981, 1031.663, 1038.582, 1046.081,
       1052.863, 1059.125, 1063.383, 1064.653, 1066.368, 1065.156,
       1064.594, 1067.451, 1069.064, 1069.9  , 1066.888, 1063.885,
       1060.84 , 1058.579, 1055.952, 1054.782, 1052.923, 1052.343,
       1053.392, 1054.808, 1055.072, 1055.525, 1054.979, 1054.672,
       1055.423, 1054.757, 1054.859, 1055.005, 1055.482, 1051.788,
       1051.012, 1049.817, 1052.496, 1059.517, 1056.668, 1055.2  ,
       1053.008, 1050.158, 1050.065, 1049.144, 1048.41 , 1046.

### 特征一致性

In [65]:
feats = df_feats.loc[selected_object_ids, X_cols]
feats

,ppg-g__ar_2,ppg-g__number_peaks__n_3,ppg-g__autocorrelation__lag_4,"ppg-g__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""var""","ppg-g__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""mean""",ppg-g__binned_entropy__max_bins_10,"ppg-g__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0","ppg-g__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8","ppg-g__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.0",ppg-g__ratio_beyond_r_sigma__r_1.5
1000,-0.108967,7.0,0.79985,0.630177,0.779862,2.103959,1.57664,1.732792,4.732611,0.132812


### 模型一致性

In [66]:
D = xgboost.DMatrix(feats.values)
bst.predict(D)

array([0.00416401], dtype=float32)

In [66]:
import json
import logging

_logger = logging.getLogger('nni')

In [67]:
_logger.info(
            "Creating graph json, writing to. Visualization enabled.")